# Ahora voy a realizar un webscrapping para conseguir datos sobre el covid en España por comunidades autónomas, casos, fallecidos, población y vacunados.

In [1]:
from pymongo import MongoClient
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import pandas as pd
import requests
import os

In [2]:
url = "https://es.wikipedia.org/wiki/Pandemia_de_COVID-19_en_Espa%C3%B1a"
res = requests.get(url)
res

<Response [200]>

In [3]:
html = BeautifulSoup(res.text)

In [13]:
tabla = html.select("div > table.sortable")

In [21]:
incidencia = tabla[0]
vacunacion = tabla[1]

# Primero iremos con la incidencia

In [46]:
incidencia.select("th")

[<th colspan="8">Incidencia de COVID-19 en España por autonomía a 7 de diciembre de 2021<sup class="reference separada" id="cite_ref-104"><a href="#cite_note-104"><span class="corchete-llamada">[</span>103<span class="corchete-llamada">]</span></a></sup>​
 </th>,
 <th>Comunidad autónoma
 </th>,
 <th>Casos
 </th>,
 <th>Fallecidos
 </th>,
 <th>Población (2020)<sup class="reference separada" id="cite_ref-Población_por_comunidades_y_ciudades_autónomas_(2020)_105-0"><a href="#cite_note-Población_por_comunidades_y_ciudades_autónomas_(2020)-105"><span class="corchete-llamada">[</span>104<span class="corchete-llamada">]</span></a></sup>​
 </th>,
 <th>Casos <abbr title="por cien mil habitantes">pcmh</abbr>
 </th>,
 <th>Fallecidos <abbr title="por cien mil habitantes">pcmh</abbr>
 </th>,
 <th style="text-align:left;"><span class="flagicon"><a href="/wiki/Andaluc%C3%ADa" title="Andalucía"><img alt="Andalucía" class="thumbborder" data-file-height="500" data-file-width="750" decoding="async" height

In [47]:
column_name=[]
for i in range(1,5):
    column_name.append((incidencia.select("th")[i]).text)

In [48]:
column_name

['Comunidad autónoma\n',
 'Casos\n',
 'Fallecidos\n',
 'Población (2020)[104]\u200b\n']

In [54]:
for i in range(0,4):
    column_name[i] = column_name[i].replace("\n","").replace(" (2020)[104]\u200b","")

In [55]:
column_name

['Comunidad autónoma', 'Casos', 'Fallecidos', 'Población']

In [60]:
rows = incidencia.select("tr")[2:21]
len(rows)

19

In [61]:
rows

[<tr>
 <th style="text-align:left;"><span class="flagicon"><a href="/wiki/Andaluc%C3%ADa" title="Andalucía"><img alt="Andalucía" class="thumbborder" data-file-height="500" data-file-width="750" decoding="async" height="13" src="//upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_Andaluc%C3%ADa.svg/20px-Flag_of_Andaluc%C3%ADa.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_Andaluc%C3%ADa.svg/30px-Flag_of_Andaluc%C3%ADa.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/2/20/Flag_of_Andaluc%C3%ADa.svg/40px-Flag_of_Andaluc%C3%ADa.svg.png 2x" width="20"/></a></span> <a href="/wiki/Andaluc%C3%ADa" title="Andalucía">Andalucía</a>
 </th>
 <td>829 848
 </td>
 <td>11 355
 </td>
 <td>8 464 411
 </td>
 <td>9803,9
 </td>
 <td>134,1
 </td></tr>,
 <tr>
 <th style="text-align:left;"><span class="flagicon"><a href="/wiki/Arag%C3%B3n" title="Aragón"><img alt="Aragón" class="thumbborder" data-file-height="667" data-file-width="1000" decoding="async" height=

In [119]:
rows = incidencia.select("tr")[2:21]
data_incidencia = [[i.get_text(strip=True) for i in row.select("th, td")] for row in rows]

In [ ]:
data_incidencia

In [120]:
data_incidencia = [[e.replace("\xa0","").replace(",",".") for e in a] for a in data_incidencia]

In [121]:
data_incidencia

[['Andalucía', '829848', '11355', '8464411', '9803.9', '134.1'],
 ['Aragón', '174138', '3971', '1329391', '13099.0', '298.7'],
 ['Asturias', '76340', '2094', '1018784', '7493.2', '205.5'],
 ['Islas Baleares', '108183', '1028', '1171543', '9234.2', '87.7'],
 ['Islas Canarias', '106189', '1061', '2175952', '4880.1', '48.7'],
 ['Cantabria', '49097', '618', '582905', '8422.8', '106.0'],
 ['Castilla-La Mancha', '246801', '6540', '2045221', '12067.2', '319.7'],
 ['Castilla y León', '315510', '7431', '2394918', '13174.1', '310.2'],
 ['Cataluña', '979986', '16041', '7780479', '12595.4', '206.1'],
 ['Ceuta', '7789', '130', '84202', '9250.3', '154.3'],
 ['Comunidad Valenciana', '542566', '7917', '5057353', '10728.2', '156.5'],
 ['Extremadura', '105668', '1999', '1063987', '9931.3', '187.8'],
 ['Galicia', '199745', '2680', '2701819', '7392.9', '99.1'],
 ['Comunidad de Madrid', '921523', '16303', '6779888', '13592.0', '240.4'],
 ['Melilla', '11444', '114', '87076', '13142.5', '130.9'],
 ['Región d

In [122]:
for datos in data_incidencia:
    for i in range(1,6):
        datos[i]=float(datos[i])

In [128]:
for datos in data_incidencia:
    datos.pop(5)
    datos.pop(4)

In [131]:
df_incidencia = pd.DataFrame(data_incidencia[:],columns=column_name)

In [132]:
df_incidencia

,Comunidad autónoma,Casos,Fallecidos,Población
0,Andalucía,829848.0,11355.0,8464411.0
1,Aragón,174138.0,3971.0,1329391.0
2,Asturias,76340.0,2094.0,1018784.0
3,Islas Baleares,108183.0,1028.0,1171543.0
4,Islas Canarias,106189.0,1061.0,2175952.0
5,Cantabria,49097.0,618.0,582905.0
6,Castilla-La Mancha,246801.0,6540.0,2045221.0
7,Castilla y León,315510.0,7431.0,2394918.0
8,Cataluña,979986.0,16041.0,7780479.0
9,Ceuta,7789.0,130.0,84202.0
